 - Aotizhongxin, Changping, Dingling (Michael) 
 - Dongsi, Guanyuan, Gucheng (David) 
 - Huairou, Nongzhanguan, Shunyi (Muareen) 
 - Tiantan, Wanliu, and Wanghouxigong (Anna)
 
 
 
 # some work to do:
  
1. Calculate AQI for each day for PMP10 and PM2.5
2. Subset the data into geographic regions
    2.1 Subset the data by seasons
3. General trend for some attributes?
        Create AIQ
        Avg number of really bad days?                        
4. Make cool looking plots/graphs with the data MAPS!
5. Analysis of those plots/graphs
    

In [1]:
# Importing required libraries.
import pandas as pd
import numpy as np
import seaborn as sns #visualisation
import matplotlib.pyplot as plt #visualisation
import seaborn as sns #visualisation
%matplotlib inline 
sns.set(color_codes=True)
import plotly.express as px
import datetime as dt

from os import listdir
from os.path import isfile, join
import glob

#conda install -c conda-forge ipyleaflet
from ipyleaflet import Map, basemaps, basemap_to_tiles, Marker, Heatmap, WidgetControl
from bqplot import Lines, Figure, LinearScale, DateScale, Axis
from ipywidgets import Dropdown

import aqi



# Data loading and cleaning methods

In [2]:
def create_date_from_string(df_to_clean, month_column, day_column, year_column):
    #add correct data column
    print('**************************')
    print('Creating date column')
    print('**************************')
    #df_to_clean['date'] = df_to_clean.apply(lambda row: str(row[month_column])+'-'+str(row[day_column])+'-'+str(row[year_column]), axis=1)
    df_to_clean['date'] = df_to_clean.apply(lambda row: dt.datetime(row[year_column], row[month_column], row[day_column]), axis=1)
    
    #df_to_clean['date_2'] =  pd.to_datetime(df_to_clean['date'])
    

In [3]:
def remove_duplicates(df_to_clean):    
    print('**************************')
    print('Removing Duplicates')
    print('**************************')
    duplicate_rows_df = df_to_clean[df_to_clean.duplicated()]
    print('number of duplicate rows: ', len(duplicate_rows_df.index))
    df_to_clean.drop_duplicates(inplace=True)    
    


In [4]:
def drop_nulls_from_df(df_to_clean, drop_rows,column_names=[]):
    if column_names == []:
        column_names == column_names.append(df_to_clean.columns)
    print('**************************')
    print('Removing Nulls')
    print('**************************') 
    
    print('- - - - - - - - - - - - - ')
    print('Before removing nulls')
    print('- - - - - - - - - - - - - ')
    print(df_to_clean.isnull().sum()) # Number of rows with nulls that we are dropping    
    
    if(drop_rows):
        df_to_clean.dropna(inplace=True, subset=column_names) 
    
    
    print('- - - - - - - - - - - - - ')
    print('After removing nulls')
    print('- - - - - - - - - - - - - ')
    print(df_to_clean.count()) #number of total rows after drop.
    


In [5]:
def clean_data(df_to_clean):
        create_date_from_string(df_to_clean, 'month', 'day', 'year')
        remove_duplicates(df_to_clean)
        drop_nulls_from_df(df_to_clean, True, ['No', 'year', 'month', 'day', 'hour', 'PM2.5', 'PM10', 'SO2', 'NO2',
       'CO', 'O3', 'TEMP', 'PRES', 'DEWP', 'RAIN', 'wd', 'WSPM', 'station', 'date'])

In [6]:
def load_csv_data(data_directory):
    #get the data files from the directory
    data_files = [f for f in listdir(data_directory) if isfile(join(data_directory, f)) and f.endswith(".csv")]
    df = pd.DataFrame() #initialize an empty dataframe
    for data_file in data_files:
        df_per_file = pd.read_csv(join(data_directory, data_file))
        #concatonate the df_weekly dataframe we just generated with the others
        df = pd.concat([df, df_per_file])

    clean_data(df)    
    return df



# Derived Values from Data

In [37]:
def calc_aqi(df_to_add_aqi):    
        # get more information from the error...
        df_to_add_aqi['aqi'] = df_to_add_aqi.apply(lambda row: aqi.to_iaqi(aqi.POLLUTANT_PM25, str(row['PM10']), algo=aqi.ALGO_EPA), axis=1)    
    

In [38]:
calc_aqi(df_all)
# test  = aqi.to_iaqi(aqi.POLLUTANT_PM10, '54', algo=aqi.ALGO_EPA)
# print(test)

IndexError: list index out of range

In [35]:
df_all

No  year  month  day  hour  PM2.5  PM10   SO2   NO2     CO  ...  \
0          1  2013      3    1     0    4.0   4.0   4.0   7.0  300.0  ...   
1          2  2013      3    1     1    8.0   8.0   4.0   7.0  300.0  ...   
2          3  2013      3    1     2    7.0   7.0   5.0  10.0  300.0  ...   
3          4  2013      3    1     3    6.0   6.0  11.0  11.0  300.0  ...   
4          5  2013      3    1     4    3.0   3.0  12.0  12.0  300.0  ...   
...      ...   ...    ...  ...   ...    ...   ...   ...   ...    ...  ...   
35059  35060  2017      2   28    19   11.0  32.0   3.0  24.0  400.0  ...   
35060  35061  2017      2   28    20   13.0  32.0   3.0  41.0  500.0  ...   
35061  35062  2017      2   28    21   14.0  28.0   4.0  38.0  500.0  ...   
35062  35063  2017      2   28    22   12.0  23.0   4.0  30.0  400.0  ...   
35063  35064  2017      2   28    23   13.0  19.0   4.0  38.0  600.0  ...   

       DEWP  RAIN   wd  WSPM        station        lat        long       date  \
0     -18.8   0.0  NNW   4.4   Aotizhongxin  23.971319  120.979889 2013-03-01   
1     -18.2   0.0    N   4.7   Aotizhongxin  23.971319  120.979889 2013-03-01   
2     -18.2   0.0  NNW   5.6   Aotizhongxin  23.971319  120.979889 2013-03-01   
3     -19.4   0.0   NW   3.1   Aotizhongxin  23.971319  120.979889 2013-03-01   
4     -19.5   0.0    N   2.0   Aotizhongxin  23.971319  120.979889 2013-03-01   
...     ...   ...  ...   ...            ...        ...         ...        ...   
35059 -16.2   0.0   NW   2.4  Wanshouxigong        NaN         NaN 2017-02-28   
35060 -15.1   0.0  WNW   0.9  Wanshouxigong        NaN         NaN 2017-02-28   
35061 -13.3   0.0   NW   1.1  Wanshouxigong        NaN         NaN 2017-02-28   
35062 -12.9   0.0  NNW   1.2  Wanshouxigong        NaN         NaN 2017-02-28   
35063 -15.9   0.0  NNE   1.3  Wanshouxigong        NaN         NaN 2017-02-28   

       season  aqi  
0      spring   17  
1      spring   17  
2      spring   17  
3      spring   17  
4      spring   17  
...       ...  ...  
35059  winter   17  
35060  winter   17  
35061  winter   17  
35062  winter   17  
35063  winter   17  

[382168 rows x 23 columns]

In [8]:
def partition_into_regions(df_to_bin):
    # define regions
    # Add new column to df 'region'
    # calculate bins
    pass

In [9]:
def partition_into_seasons(df_to_bin,month_column):    
    #https://seasonsyear.com/China
    #summer = [6,7,8]
    #fall = [9,10,11]
    #winter = [12,1,2]
    #spring = [3,4,5]    
    # define seasons
    # Add new column to df 'season'
    # calculate bins
    #hacky....but....fast...
    seasons = ['winter','winter','spring','spring','spring', 'summer', 'summer', 'summer','fall','fall', 'fall','winter']    
    df_to_bin['season'] = df_to_bin.apply(lambda row: seasons[row[month_column]-1], axis=1)
    

In [10]:
def calculte_average_of_bad_days(df_to_average):
    #Running average of 7 days? for the two predictors
    #should return a new dataframe?
    pass

In [11]:
#actually laod all the data, clean it, and put it into a dataframe
df_all = load_csv_data("data")

#run all of our calculations
# calc_aiq_for_XX(df_all, 'XXX')
# partition_into_regions(df_all)
partition_into_seasons(df_all,'month')
# df_moving_average = calculte_average_of_bad_days(df_all)


**************************
Creating date column
**************************
**************************
Removing Duplicates
**************************
number of duplicate rows:  0
**************************
Removing Nulls
**************************
- - - - - - - - - - - - - 
Before removing nulls
- - - - - - - - - - - - - 
No              0
year            0
month           0
day             0
hour            0
PM2.5        8739
PM10         6449
SO2          9021
NO2         12116
CO          20701
O3          13277
TEMP          398
PRES          393
DEWP          403
RAIN          390
wd           1822
WSPM          318
station         0
lat        210384
long       210384
date            0
dtype: int64
- - - - - - - - - - - - - 
After removing nulls
- - - - - - - - - - - - - 
No         382168
year       382168
month      382168
day        382168
hour       382168
PM2.5      382168
PM10       382168
SO2        382168
NO2        382168
CO         382168
O3         382168
TEMP       38

In [ ]:
#quick list of all unique station names
station_names = df_all.station.unique()
station_names

In [ ]:
#Select just one station to examine
#df = df_all[df_all['station'].isin(['Aotizhongxin','Changping','Dingling'])]
df_A = df_all[df_all['station'].isin(['Aotizhongxin'])]
print('length of A : {}'.format(len(df_A.index)))
df_C = df_all[df_all['station'].isin(['Changping'])]
print('length of C : {}'.format(len(df_C.index)))
df_D = df_all[df_all['station'].isin(['Dingling'])]
print('length of D : {}'.format(len(df_D.index)))    

# Some exploration

lat longs:

- Aotizhongxin :  (41.741127, 123.462775)
- Changping :  (40.220585, 116.228038)
- Dingling :  (40.289968, 116.237352)
- Dongsi :  (39.929855, 116.421619)
- Guanyuan :  (39.932482, 116.355741)
- Gucheng :  (39.907599, 116.190328)
- Huairou :  (40.605853, 116.622746)
- Nongzhanguan :  (39.945631, 116.475666)
- Shunyi :  (40.152315, 116.714525)
- Tiantan :  (39.888430, 116.409856)
- Wanliu :  (39.977951, 116.292273)
- Wanghouxigong :  (,)


In [ ]:
df_numeric = df_all[['PM2.5','PM10','SO2','NO2','CO','O3','TEMP','PRES','DEWP','RAIN']]
sns.boxplot(x="variable", y="value", data=pd.melt(df_numeric))

In [ ]:
Q1 = df_numeric.quantile(0.25)
Q3 = df_numeric.quantile(0.75)
IQR = Q3-Q1
print(IQR)

In [ ]:
# Plotting a Histogram
df_all.CO.value_counts().nlargest(40).plot(kind='bar', figsize=(10,5))
plt.title('Number of days by CO')
plt.ylabel('Number of days')
plt.xlabel('CO');

In [ ]:
# Timeseries
fig = px.line(df_A, x='date', y='CO')
# fig.add_scatter(x=df_all['date'], y=df_all['PM10'], mode='lines')
# fig.add_scatter(x=df_all['date'], y=df_all['SO2'], mode='lines')
#fig.add_scatter(x=df_all['date'], y=df_all['O3'], mode='lines')         

fig.show()

In [ ]:
df_all.PM10.value_counts().nlargest(40).plot(kind='bar', figsize=(10,5))
plt.title('Number of days by PM10')
plt.ylabel('Number of days')
plt.xlabel('PM10');

In [ ]:
# Finding the relations between the variables.
plt.figure(figsize=(20,10))
c= df_all.corr()
sns.heatmap(c,cmap='BrBG',annot=True)
c

In [ ]:
# Plotting a scatter plot
fig, ax = plt.subplots(figsize=(10,6))
ax.scatter(df_all['NO2'], df_all['WSPM'])
ax.set_xlabel('NO2')
ax.set_ylabel('WSPM')
plt.show()

# Map Stuff

In [ ]:
def get_on_hover(marker):        
    def callback(*args, **kwargs):        
        global station_name
        station_name = marker.name
        print(station_name)
        update_figure(station_name, data_name)
    return callback

In [ ]:
from bqplot import Lines, Figure, LinearScale, DateScale, Axis

#some globals...not exactly the best way to do this buuut..
global station_name
global year
global season
global data_name

station_name = 'Aotizhongxin'
year = 2015
season = 'summer'
data_name = 'CO'



#x_data = df_all[df_all.station == station_name and df_all.year == year and df_all.season == season][data_name].values
y_data = df_all[(df_all['season'] == season) & (df_all['station'] == station_name) & (df_all['year'] == year)]['CO'].values
x_data = df_all[(df_all['season'] == season) & (df_all['station'] == station_name) & (df_all['year'] == year)]['date'].values

x_scale = LinearScale()

date_start = dt.datetime(year, 1, 1)
date_end = dt.datetime(year, 12, 31)

date_scale = DateScale(min=date_start, max=date_end)

lines = Lines(x=x_data, y=y_data ,scales={'x': date_scale, 'y': x_scale})
lines

ax_x = Axis(label='Year', scale=date_scale, num_ticks=10, tick_format='%y')
ax_y = Axis(label=data_name.capitalize(), scale=x_scale, orientation='vertical', side='left')

figure = Figure(axes=[ax_x, ax_y], title=station_name, marks=[lines], animation_duration=500,
                 layout={'max_height': '250px', 'max_width': '400px'})


figure

In [ ]:
#update_figure('Aotizhongxin', 'CO', year=2013, season='spring')
#update_figure('Aotizhongxin', 'CO', year=2013, season='summer')
#update_figure('Aotizhongxin', 'CO', year=2013, season='fall')
#update_figure('Aotizhongxin', 'CO', year=2013, season='winter')

update_figure('Aotizhongxin', 'CO', 2013, 'fall')

In [ ]:
m = Map(center=(39.987916, 116.383936), zoom=12)

stations = {
    'Aotizhongxin' : (39.987916, 116.383936),
    'Changping' : (40.220585, 116.228038),
    'Dingling' : (40.289968, 116.237352),
    'Dongsi' : (39.929855, 116.421619),
    'Guanyuan' : (39.932482, 116.355741),
    'Gucheng' : (39.907599, 116.190328),
    'Huairou' : (40.605853, 116.622746),
    'Nongzhanguan' : (39.945631, 116.475666),
    'Shunyi' : (40.152315, 116.714525),
    'Tiantan' : (39.888430, 116.409856),
    'Wanliu' : (39.977951, 116.292273)
}

for station in stations.items():
    marker = Marker(location=station[1], draggable=False, title=station[0], name=station[0])
    marker.on_mouseover(get_on_hover(marker))
    m.add_layer(marker);    
    #m.add_layer(Marker(location=station[1], draggable=False));

# marker = Marker(location=center, draggable=False)
# m.add_layer(marker);

map_layer = basemap_to_tiles(basemaps.CartoDB.Positron)
m.add_layer(map_layer)
m


In [ ]:
widget_control1 = WidgetControl(widget=figure, position='bottomright')
m.add_control(widget_control1)

In [ ]:
def update_figure(station_name, data_name, year, season):
    
    year_start = 2013
    year_end = 2016
    
    if(year == 0 and season == ''):
        y_data = df_all[(df_all['season'] == season) & (df_all['station'] == station_name) & (df_all['year'] == year)][data_name].values
        x_data = df_all[(df_all['season'] == season) & (df_all['station'] == station_name) & (df_all['year'] == year)]['date'].values
        
        year_start = year
        year_end = year
        
    else :
        y_data = df_all[(df_all['station'] == station_name)][data_name].values
        x_data = df_all[(df_all['station'] == station_name)]['date'].values
           
    lines.y = y_data
    lines.x = x_data
    
    ax_y.label = data_name.capitalize()
    figure.title = station_name
    
    date_start = dt.datetime(year_start, 1, 1)
    date_end = dt.datetime(year_end, 12, 31)
    date_scale = DateScale(min=date_start, max=date_end)
    lines.scales={'x': date_scale, 'y': x_scale}
    

In [ ]:
dropdown = Dropdown(
    options=['CO', 'PM10', 'PM2.5'],
    value=data_name,
    description='Plotting:'
)

def data_on_click(change):
    global data_name
    data_name = change['new']
    update_figure(station_name, data_name, 2013, 'winter')

dropdown.observe(data_on_click, 'value')

widget_control2 = WidgetControl(widget=dropdown, position='bottomleft')

m.add_control(widget_control2)



In [ ]:
dropdown_year = Dropdown(
    options=['2013', '2014', '2015', '2016'],
    value=year,
    description='Plotting:'
)

def year_on_click(change):
    global data_year
    year = change['new']
    update_figure(station_name, data_name, year)

dropdown_year.observe(year_on_click, 'value')

widget_control_year = WidgetControl(widget=dropdown_year, position='bottomleft')

m.add_control(widget_control_year)


In [ ]:
dropdown_season = Dropdown(
    options=['spring', 'summer', 'winter', 'fall'],
    value=season,
    description='Plotting:'
)

def data_on_click(change):
    global data_name
    data_name = change['new']
    update_figure(station_name, data_name)

dropdown.observe(data_on_click, 'value')

widget_control2 = WidgetControl(widget=dropdown, position='bottomleft')

m.add_control(widget_control2)


In [ ]:
y_data = df_all[(df_all['season'] == %) & (df_all['station'] == 'Aotizhongxin') & (df_all['year'] == 2016)]['CO'].values
y_data